In [1]:
import torch
torch.cuda.empty_cache() 

In [2]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image

In [3]:
import pandas as pd
import pyproj

In [4]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [5]:
from pathlib import Path

In [6]:
import joblib

In [7]:
from mt_function_yl import *

In [8]:
import seaborn as sn

In [9]:
from scipy import stats

In [10]:
from tabulate import tabulate

In [11]:
import statsmodels.api as sm

### Data loading

In [12]:
aa = pd.read_csv('/home/yelhe/script/mt/dataraw/dfaa.csv')

In [13]:
atime = aa[['Nr', 'Datum', 'Zeit']]

In [14]:
atime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Nr      350 non-null    int64 
 1   Datum   350 non-null    object
 2   Zeit    350 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.3+ KB


In [15]:
atime

,Nr,Datum,Zeit
0,99900060852597,2022-05-29,00:10
1,99900063018611,2022-10-12,17:40
2,99900052723510,2020-10-26,16:50
3,99900061514043,2022-07-08,21:30
4,99900063037843,2022-10-13,19:40
...,...,...,...
345,99900055905276,2021-06-08,12:50
346,99900060670975,2022-05-16,14:42
347,99900047367168,2019-09-21,13:49
348,99900050714619,2020-06-06,16:37


In [18]:
atime.loc[:, 'Datum'] = pd.to_datetime(atime.loc[:, 'Datum'], format = "%Y-%m-%d")

In [22]:
atime.loc[:,'year'] = atime.loc[:,'Datum'].dt.year
atime.loc[:,'month'] = atime.loc[:,'Datum'].dt.month
atime.loc[:,'day'] = atime.loc[:,'Datum'].dt.day

In [23]:
atime

,Nr,Datum,Zeit,year,month,day
0,99900060852597,2022-05-29,00:10,2022,5,29
1,99900063018611,2022-10-12,17:40,2022,10,12
2,99900052723510,2020-10-26,16:50,2020,10,26
3,99900061514043,2022-07-08,21:30,2022,7,8
4,99900063037843,2022-10-13,19:40,2022,10,13
...,...,...,...,...,...,...
345,99900055905276,2021-06-08,12:50,2021,6,8
346,99900060670975,2022-05-16,14:42,2022,5,16
347,99900047367168,2019-09-21,13:49,2019,9,21
348,99900050714619,2020-06-06,16:37,2020,6,6


In [26]:
atime.loc[:,'week'] = atime.loc[:,'Datum'].dt.isocalendar().week

In [28]:
atimegroup = atime.groupby(['year']).size().reset_index().rename(columns={0:'count'})

In [69]:
atimegroup

,year,count
0,2019,48
1,2020,74
2,2021,91
3,2022,137


In [32]:
# Traffic Count bicycle
btc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2019.csv')
btc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2020.csv')
btc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2021.csv')
# btc22

# Traffic count car
vtc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2019.csv')
vtc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2020.csv')
vtc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2021.csv')
# vtc22

In [39]:
btc1921 = pd.concat([btc19, btc20, btc21])
vtc1921 = pd.concat([vtc19, vtc20, vtc21])

In [41]:
ac_btc1921 = atime.merge(btc1921, on = ['year', 'month', 'week'])
ac_vtc1921 = atime.merge(vtc1921, on = ['year', 'month', 'week'])

In [44]:
# Shortest distance file
disb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_bicyclecount.csv') 
disv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_carcount.csv') 

In [64]:
ac_vtc1921.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34216 entries, 0 to 34215
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Nr                 34216 non-null  int64         
 1   Datum              34216 non-null  datetime64[ns]
 2   Zeit               34216 non-null  object        
 3   year               34216 non-null  int32         
 4   month              34216 non-null  int32         
 5   day                34216 non-null  int32         
 6   week               34216 non-null  UInt32        
 7   MSID               34216 non-null  object        
 8   car_traffic_count  34216 non-null  int64         
 9   date               34216 non-null  object        
 10  OST                34216 non-null  float64       
 11  NORD               34216 non-null  float64       
 12  car_traffic_norm   34216 non-null  float64       
dtypes: UInt32(1), datetime64[ns](1), float64(3), int32(3), int64(

In [63]:
disvu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Nr        355 non-null    int64  
 1   MSID      355 non-null    object 
 2   distance  355 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 8.4+ KB


In [53]:
disbu = disb[['Nr', 'fk_zaehler', 'distance']]
disbu = disbu.drop_duplicates()

In [57]:
disvu = disv[['Nr', 'MSID', 'distance']]
disvu = disvu.drop_duplicates()

In [62]:
disbu = disbu.rename(columns = {'fk_zaehler': 'FK_ZAEHLER'})

In [65]:
acc_bc1921 = ac_btc1921.merge(disbu, on = ['Nr','FK_ZAEHLER'])
acc_vc1921 = ac_vtc1921.merge(disvu, on = ['Nr', 'MSID'])

In [68]:
print(len(acc_bc1921['Nr'].unique()), len(acc_vc1921['Nr'].unique()))

146 188


In [74]:
len(ac_btc1921['FK_ZAEHLER'].unique())

58

In [73]:
len(disbu['FK_ZAEHLER'].unique())

70

In [78]:
acc_bc1921.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount1921.csv')
acc_vc1921.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount1921.csv')

In [76]:
acc_bc1921.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Nr                     155 non-null    int64         
 1   Datum                  155 non-null    datetime64[ns]
 2   Zeit                   155 non-null    object        
 3   year                   155 non-null    int32         
 4   month                  155 non-null    int32         
 5   day                    155 non-null    int32         
 6   week                   155 non-null    UInt32        
 7   FK_ZAEHLER             155 non-null    object        
 8   bicycle_traffic_count  155 non-null    float64       
 9   date                   155 non-null    object        
 10  OST                    155 non-null    int64         
 11  NORD                   155 non-null    int64         
 12  bicycle_traffic_norm   155 non-null    float64       
 13  dista

In [77]:
acc_bc1921['year'].unique()

array([2020, 2021, 2019], dtype=int32)